# 預測「未來 dayforesee 天會不會漲 20%？」

實際 4638天當中有 __ 天符合條件


In [1]:
%%js
alert("要不要把 model/DNN directory 先殺掉？")


<IPython.core.display.Javascript object>

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import peforth as f
from sklearn import cross_validation

c:\users\hcche\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


reDef unknown
reDef \


In [4]:
data = pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

# data frame 中帶有 None 的 rows 都剔除掉。 本來有 4752 rows 現在只剩 4639 
# 用法: df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data = data.replace(to_replace='None', value=np.nan).dropna()  

# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

# 上市第一天的資料不要，因為【漲跌幅】無效。
data = data.query('index > "1997-07-10"')

In [5]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature=150
dayforesee=10  # 問未來幾天如何如何，最後就要扣掉幾天，避免 out of bounds. 0 代表當天。
featurenum=5*dayfeature  # 取五個有關係的欄位，特徵值。
x=np.zeros((data.shape[0]-dayfeature-dayforesee,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature-dayforesee));

In [6]:
# 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    # x[i,featurenum]=data.ix[i+dayfeature][u'开盘价']  # ix deprecated
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']

In [7]:
# 預測未來 dayforesee 天是否會「漲20%」?

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    y[i]=0              
    pi = float(data.iloc[i][u'收盘价'])
    for j in range(dayforesee):
        pi5 = float(data.iloc[j+i+dayfeature][u'最高价'])
        if (pi5-pi)/pi >= 0.20 :
            y[i]=1


In [8]:
y = np.int32(y)
y

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2)

In [10]:
%f ( 有這麼多天符合條件 ) y py> sum(pop()) tib.
%f x_train :> shape tib.
%f x_test :> shape tib.
%f y_train :> shape tib.
%f y_test :> shape tib.
%f x_train :> shape[0] tib.
    

( 有這麼多天符合條件 ) y py> sum(pop()) tib. \ ==> 1677 (<class 'numpy.int32'>)
x_train :> shape tib. \ ==> (3582, 751) (<class 'tuple'>)
x_test :> shape tib. \ ==> (896, 751) (<class 'tuple'>)
y_train :> shape tib. \ ==> (3582,) (<class 'tuple'>)
y_test :> shape tib. \ ==> (896,) (<class 'tuple'>)
x_train :> shape[0] tib. \ ==> 3582 (<class 'int'>)


In [11]:
y_train

array([0, 1, 0, ..., 1, 0, 1])

In [12]:
# 應該只有一個 feature 是個 shape = (751,) 的 array. 這怎麼表達?
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[751,])]  
my_feature_columns

[_NumericColumn(key='x', shape=(751,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [13]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[256,128],
    # The model must choose between 3 classes.
    n_classes=2,
    dropout=0.4,
    model_dir="model/DNN"
    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/DNN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000245DC10B1D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
# Define the training inputs 從 mnist 抄過來改的
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    num_epochs=None,
    batch_size=1000,
    shuffle=True
    )

In [15]:
classifier.train(input_fn=train_input_fn, steps=3000);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into model/DNN\model.ckpt.
INFO:tensorflow:loss = 1706707600.0, step = 1
INFO:tensorflow:global_step/sec: 36.4018
INFO:tensorflow:loss = 25318508.0, step = 101 (2.749 sec)
INFO:tensorflow:global_step/sec: 37.109
INFO:tensorflow:loss = 8594579.0, step = 201 (2.696 sec)
INFO:tensorflow:global_step/sec: 36.1119
INFO:tensorflow:loss = 1974630.0, step = 301 (2.768 sec)
INFO:tensorflow:global_step/sec: 30.6263
INFO:tensorflow:loss = 894211.6, step = 401 (3.268 sec)
INFO:tensorflow:global_step/sec: 32.3098
INFO:tensorflow:loss = 888675.2, step = 501 (3.092 sec)
INFO:tensorflow:global_step/sec: 33.619
INFO:tensorflow:loss = 313553.72, step = 601 (2.976 sec)
INFO:tensorflow:global_step/sec: 34.7729
INFO:tensorflow:loss = 

In [16]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
    )

In [17]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

# End of "mnist_estimator.py"


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-03-08:37:37
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-03-08:37:38
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.62053573, accuracy_baseline = 0.60491073, auc = 0.6149124, auc_precision_recall = 0.5510735, average_loss = 580.3221, global_step = 3000, label/mean = 0.3950893, loss = 74281.23, precision = 0.5421687, prediction/mean = 0.44034973, recall = 0.2542373

Test Accuracy: 62.053573%



### 用全部資料來比對

In [18]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x},
    y=y,
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [19]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [20]:
predict = [i for i in preds];

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [21]:
predictions = [int(i['probabilities'][1] > i['probabilities'][0]) for i in predict];

In [22]:
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.
%f y :> [:50] . cr


predictions py> sum(pop()) tib. \ ==> 817 (<class 'int'>)
y py> sum(pop()) tib. \ ==> 1677 (<class 'numpy.int32'>)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [23]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(y.shape[0]): 
    if int(predictions[i])==int(y[i]) and int(y[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y[i]) and int(y[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

categories ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [2418, 1243, 383, 434] (<class 'list'>)


### 只用 Test dataset 來比對

In [24]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [25]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [26]:
predict = [i for i in preds];

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/DNN\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [27]:
predictions = [int(i['probabilities'][1] > i['probabilities'][0]) for i in predict];

In [28]:
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.
%f y :> [:50] . cr


predictions py> sum(pop()) tib. \ ==> 166 (<class 'int'>)
y py> sum(pop()) tib. \ ==> 1677 (<class 'numpy.int32'>)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [29]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(y_test.shape[0]): 
    if int(predictions[i])==int(y[i]) and int(y[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y[i]) and int(y[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y[i]) and int(y[i])==1: categories[3]+=1; continue
%f categories ( 沒事的，漏掉的，多的，一致的 ) tib.

categories ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [433, 297, 110, 56] (<class 'list'>)


In [30]:
!start tensorboard --logdir="model/DNN"

# 都搞懂了，接下來打包 000777.csv 試試看吧!

這把預測「明天會不會漲停？」結果因為實際 4638天當中只有88天漲停，全部猜「不」就有 98.10％ 的 accuracy. 可能正是因為怎麼猜別的都沒有這個好。 DNN 全部猜「不會」。


In [ ]:
%%js
alert("要不要把 DNN_model directory 先殺掉？")


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import peforth as f
from sklearn import cross_validation

In [ ]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


In [ ]:
data = pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

# data frame 中帶有 None 的 rows 都剔除掉。 本來有 4752 rows 現在只剩 4639 
# 用法: df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data = data.replace(to_replace='None', value=np.nan).dropna()  

# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

# 上市第一天的資料不要，因為【漲跌幅】無效。
data = data.query('index > "1997-07-10"')

In [ ]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature=150
dayforesee=5  # 問未來幾天如何如何，最後就要扣掉幾天，避免 out of bounds. 0 代表當天。
featurenum=5*dayfeature  # 取五個有關係的欄位，特徵值。
x=np.zeros((data.shape[0]-dayfeature-dayforesee,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature-dayforesee));

In [ ]:
data.shape

In [ ]:
# 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    # x[i,featurenum]=data.ix[i+dayfeature][u'开盘价']  # ix deprecated
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']

In [ ]:
# 預測當天是否會「漲停」?

# data.shape[0]-dayfeature 是 4489 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4488, 751)
# dayfeature 是 150 
# 直接看【涨跌幅】> 9.8 就算漲停板

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    if float(data.iloc[i+dayfeature][u'涨跌幅'])>=9.8:
        y[i]=1.0
    else:
        y[i]=0.0          

In [ ]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2)

In [ ]:
%f x_train :> shape tib.
%f x_test :> shape tib.
%f y_train :> shape tib.
%f y_test :> shape tib.
%f x_train :> shape[0] tib.
    

用 cross_validation 切分好 train-test feature & label 之後, 可望弄成 input_fn 要的形式

In [ ]:
# 應該只有一個 feature 是個 shape = (751,) 的 array. 這怎麼表達?
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[751,])]  
my_feature_columns

In [ ]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[256, 128],
    # The model must choose between 3 classes.
    n_classes=2,
    dropout=0.2,
    model_dir="DNN_model"
    )

以上 classifier 弄出來了，還是要面對 input_function 怎麼弄的問題。Lambda 太難懂了，從 wh300 或哪裡去抄吧。
看到的都是 tf.estimator.inputs.pandas_input_fn 我曾查出有 'numpy_input_fn' 

    c:\Users\hcche\Documents\GitHub\twstock\mnist_estimator.ipynb 
    # Specify feature
    feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]
    幹！ 回想起來了，用這個例子可以簡化 150天 的資料製作方式

Estimator 的 DNNClassifier 怎麼用? 上面這個 mnist 也有似乎更進一步的發揮

    # Build 2 layer DNN classifier
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        hidden_units=[256, 32],
        optimizer=tf.train.AdamOptimizer(1e-4),
        n_classes=10,
        dropout=0.1,
        model_dir="./tmp/mnist_model"
        )
    

In [ ]:
# Define the training inputs 從 mnist 抄過來改的
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    num_epochs=None,
    batch_size=x_train.shape[0],  # 漲停很少，故 batch 要用全部天數，否則它全部猜「不會」
    shuffle=True
    )

In [ ]:
classifier.train(input_fn=train_input_fn, steps=1000);

In [ ]:
# Define the test inputs  從 mnist 抄過來改的
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
    )

In [ ]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

# End of "mnist_estimator.py"


讚！ 接下來怎麼看 predict 的結果 . . . 參考 wh300 吧


In [ ]:
# Make input function for prediction:
#   shuffle=False -> do not randomize input data
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x},
    y=y,
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [ ]:
# 從 wh300/DNNClassifier_Predicting.ipynb 抄來的例子
# Predict 整批 test input 
preds = classifier.predict(pred_input_fn)

In [ ]:
predictions = [int(i['probabilities'][1] > i['probabilities'][0]) for i in preds];

In [ ]:
%f predictions py> sum(pop()) tib.
%f y py> sum(pop()) tib.


# 我在 Tensorflow 官網的教學網頁上找到了用 Iris 當例題的 estimator tutorial 

https://www.tensorflow.org/versions/master/get_started/premade_estimators (also on my Ynote "Get Started with Estimators _TensorFlow_") 有正式、詳細的介紹。
這篇文章鼓勵人用 Estimator API 跟 Dataset API。本文研讀、實驗這篇文章。
文章引用的 GitHub resource 我已經 clone 下來了在 T550 `c:\Users\hcche\Documents\GitHub\models`  

重點是，直接用 pandas data-frame 來當作 feature - label 餵給 input function 這樣更自然，免去搞 150 天行情資料的麻煩。


In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd

In [ ]:
import peforth

In [ ]:
%%f
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.

: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;

\ set path for 'import iris_data'
path-to-find-modules c:\Users\hcche\Documents\GitHub\models\samples\core\get_started

code \ print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


In [ ]:
iris = datasets.load_iris()

In [ ]:
iris.feature_names

In [ ]:
iris.data[:4]

In [ ]:
# df = pd.DataFrame(np.array(irisX).reshape(-1,4), columns = iris.feature_names)    
x = pd.DataFrame(iris.data, columns = iris.feature_names)    
x[:4]

In [ ]:
y = pd.DataFrame(iris.target, columns = ["target"])    
y[48:52]

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn

tf.estimator.inputs.pandas_input_fn(
    x,
    y,
    batch_size=64,
    num_epochs=5,
    shuffle=False,
    queue_capacity=None,
    num_threads=1,
    target_column='target'
)

# 照 Tutorial 執行 -- 一把就成功！

直接把本 notebook 建個 hard link 到 c:\Users\hcche\Documents\GitHub\models\samples\core\get_started 去執行...


In [ ]:
%run premade_estimator.py

## 開始照教材研讀
    
premade_estimator.py iris_data.py 都抓過來，他本來都放在 main() 內執行，不利 study 今予拆開，分段執行亦可，如下：

In [ ]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""An Example of a DNNClassifier for the Iris dataset."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import tensorflow as tf

import iris_data  # iris_data.py 在本目錄下 GitHub\models\samples\core\get_started\iris_data.py 

這 argparse module 很常見 DeepSpeech 就有，這裡用不著吧！

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')

In [ ]:
# 不能在 jupyter notebook 下直接跑，因為 argv 不如預期。
from IPython.display import display,Image;display(Image('error1.jpg', width=700))

#### Command Line Arguments 要用模擬的
Module argparse 只能在 python command line 跑

In [ ]:
%f \ [x] 沒有指定時，自動取 default 值，真不知道這哪來的？看到了，從 parser 那邊 code 裡給的。
args = parser.parse_args(["--batch_size=100", "--train_steps=1000"])
args

#### 研究  (train_x, train_y), (test_x, test_y) = iris_data.load_data()

看出來了 iris_data.py 是把 150 筆資料切成 120 : 30 for training and testing respectively.
這是個壞消息，他沒有像 sklearn 的自動切的那啥 function. 

In [ ]:
 (train_x, train_y), (test_x, test_y) = iris_data.load_data()

In [ ]:
%f train_x type . cr
train_x

In [ ]:
%f train_y type . cr
train_y

In [ ]:
%f test_x type . cr
test_x

In [ ]:
%f test_y type . cr
test_y

In [ ]:
%f \ [ ] 這是 feature columns, 我晚點可以嘗試把它弄成光一個 shape 為 (4,) 的單一 feature 
for key in train_x.keys():
    print(key)

In [ ]:
%f \ [ ] 這是 feature columns, 我晚點可以嘗試把它弄成光一個 shape 為 (4,) 的單一 feature 
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
my_feature_columns    

In [ ]:
%f \ 從 main 裡拆分出來，方便 tracing. 可看出若無指定 model directory 自動放在
%f \     AppData\Local\Temp\tmpijrg8mkm <-- 每次都不一樣，這樣好嗎？
%f \ 以及本 classifier 的種種細節
%f \ 

# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3)

In [ ]:
%f \ 複習一下 lambda --> 
%f \ lambda 是個 function, 產生 key(i) 取 x 的第 i 個 element。用於 sorted() 時指定哪個當 key 
s = [('a', 3), ('b', 2), ('c', 1)]
key = lambda x:x[1]
print(sorted(s, key=key))
%f \ 單獨使用時，看起來不太一樣，其實做的是一樣的事情。
%f key :> ([('a',3),('b',2),('c',1),('d',4)]) tib.
%f key tib.
%f key .source
%f \ 當沒有給 lambda argument 時，是怎樣？
x = lambda:s
%f x tib. \ 直接傳回 : 之後的東西(執行後)的傳回值
%f x type tib.
%f \ 查看 x 的 source code 
%f x .source
%f x :> () tib.

%f \ 所以，input_fn = lambda:iris_data.train_input_fn(train_x, train_y,args.batch_size)
%f \ 當中的 iris_data.train_input_fn() 是傳回一個 function 但是該 function 的 argument 是活的，
%f \ 要延後到最後一秒鐘才確定，因此要用 lambda 或單純的 function 包一層用來指定 arguments。
    

In [ ]:
%f \ 這就開始訓練了 . . .  
%f \ 

# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y,
                                             args.batch_size),
    steps=args.train_steps)

這時候的 classifier 已經受過訓練了! 可以接受考核了

In [ ]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:iris_data.eval_input_fn(test_x, test_y,
                                            args.batch_size))

In [ ]:
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [ ]:
%f \ 複習一下 **eval_result 啥意思？
%f eval_result . cr
%f eval_result :> ['accuracy'] . cr
{**eval_result}

In [ ]:
rslt = {'accuracy': 0.96666664, 'average_loss': 0.056607112, 'loss': 1.6982133, 'global_step': 1000}
print('Test set accuracy: {loss:0.3f}'.format(**rslt))
print('Test set accuracy: {:0.3f}'.format(r['accuracy']))
print('Test set accuracy: {a:0.3f}'.format(**{'a': 11,'b': 22}))
print('Test set accuracy: {b:0.3f}'.format(**{'a': 33,'b': 44}))
%f \ 最後一式拆解不成功，拆開來的寫法也沒意義，要嘛直接寫就好了。
%f \ print('Test set accuracy: {b:0.3f}'.format(({'a': 55},{'b': 66})))

In [ ]:
# 這一段的意思是, 準備了 3 或 4 組資料 predict_x 分別的正確答案如 expected 所列
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica','Versicolor']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9,6.2],
    'SepalWidth': [3.3, 3.0, 3.1,2.9],
    'PetalLength': [1.7, 4.2, 5.4,4.3],
    'PetalWidth': [0.5, 1.5, 2.1,1.3],
}

[(i,j) for i,j in zip(predict_x,expected)]
[i for i in zip(predict_x,expected)]


In [ ]:
predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=args.batch_size))

In [ ]:
# 這樣寫也可以
predictions = classifier.predict(
    lambda:iris_data.eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=args.batch_size))

In [ ]:
%f predictions dir . cr

In [ ]:
%f \ 以上 predictions 瞬間就取得了
%f predictions tib.
%f \ 這 'generator' 用過就沒了，要再看一遍就得重新跑一次 classifier.predict()
pred = [i for i in predictions]
pred

In [ ]:
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

In [ ]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

In [ ]:
zip?
%f \ zip() 傳回 iterator 像拉拉鍊一樣，所有第一個一組、所有第二個一組、如此以往到哪個先結束為止
z = zip((1,2,3,4),(11,22,33),{'aa':1,'bb':2})
%f z type tib.
z1 = [i for i in z]
z2 = [i for i in z]
%f z1 tib.
%f z2 tib.

In [ ]:
[(pred_dict, expec) for pred_dict, expec in zip(pred, expected)]

所以賣弄這個方法時 pred （測試資料）的個數與 expected （標準結果）就要吻合才有意思。以下簡單就是 prediction 與標準答案的比對列出來看。

In [ ]:
for pred_dict, expec in zip(pred, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

# 以下是原本的教學程式

### premade_estimator.py
c:\Users\hcche\Documents\GitHub\models\samples\core\get_started\premade_estimator.py 

In [ ]:
def main(argv):
    args = parser.parse_args("")  # default is (["--batch_size=100", "--train_steps=1000"])
    # Fetch the data
    (train_x, train_y), (test_x, test_y) = iris_data.load_data()

    # Feature columns describe how to use the input.
    my_feature_columns = []
    for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        # Two hidden layers of 10 nodes each.
        hidden_units=[10, 10],
        # The model must choose between 3 classes.
        n_classes=3)

    # Train the Model.
    classifier.train(
        input_fn=lambda:iris_data.train_input_fn(train_x, train_y,
                                                 args.batch_size),
        steps=args.train_steps)

    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda:iris_data.eval_input_fn(test_x, test_y,
                                                args.batch_size))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    # Generate predictions from the model
    expected = ['Setosa', 'Versicolor', 'Virginica']
    predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }

    predictions = classifier.predict(
        input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                                labels=None,
                                                batch_size=args.batch_size))

    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    for pred_dict, expec in zip(predictions, expected):
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(iris_data.SPECIES[class_id],
                              100 * probability, expec))


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run(main)

### iris_data.py
c:\Users\hcche\Documents\GitHub\models\samples\core\get_started\iris_data.py

In [ ]:
import pandas as pd
import tensorflow as tf

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

# 這個有用到
def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

# 這個有用到
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


# 這個有用到
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset


# The remainder of this file contains a simple example of a csv parser,
#     implemented using the `Dataset` class.

# `tf.parse_csv` sets the types of the outputs to match the examples given in
#     the `record_defaults` argument.
CSV_TYPES = [[0.0], [0.0], [0.0], [0.0], [0]]

def _parse_line(line):
    # Decode the line into its fields
    fields = tf.decode_csv(line, record_defaults=CSV_TYPES)

    # Pack the result into a dictionary
    features = dict(zip(CSV_COLUMN_NAMES, fields))

    # Separate the label from the features
    label = features.pop('Species')

    return features, label


def csv_input_fn(csv_path, batch_size):
    # Create a dataset containing the text lines.
    dataset = tf.data.TextLineDataset(csv_path).skip(1)

    # Parse each line.
    dataset = dataset.map(_parse_line)

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


    ----- play ground -----

In [ ]:
# This cell creates %%csv cell magic. See http://ipython-books.github.io/14-creating-an-ipython-extension-with-custom-magic-commands
import pandas as pd
from io import StringIO
from IPython.core.magic import (register_cell_magic)
@register_cell_magic
def csv(line, cell):
    return pd.read_csv(StringIO(cell), sep=line, engine='python')

In [ ]:
%%csv \s+
a b c
1 2 3
4 5 6
7 8 9

In [ ]:
%f words module
%f help modul

In [ ]:
%f none value locals
def smallworld():
    x = 11
    y = 22
    peforth.push(locals()).dictate('to locals').ok

In [ ]:
peforth.ok()    

In [ ]:
input_fn=lambda:iris_data.train_input_fn(train_x, train_y, args.batch_size)
%f \ lambda:iris_data.train_input_fn(_x, _y, _size)  # 定義時 identifiers 可以亂寫,那這好像是 macro 了?
%f input_fn constant i/f
%f i/f . cr
%f i/f type . cr
%f i/f dir . cr
%f i/f :> () . cr

In [ ]:
f = lambda:locals

def outer():
    x = '1'
    y = '2'
    
    def given(locs):
        return(locs)
    
    def getit(locs=locals()):
        return(locs) 
    
    def getit2(locs=locals):
        return(locs()) 
    
    def testgiven():
        x = 33
        y = 44
        l = given(locals())
        return(x,y,l)
    
    def testgetit():
        x = 55
        y = 66
        l = getit()
        return(x,y,l)
    
    def testgetit2():
        x = 77
        y = 88
        l = getit2()
        return(x,y,l)

    return testgiven(), testgetit(), testgetit2()

outer()

In [ ]:
f = input_fn()

In [ ]:
f

In [ ]:
def myDebugger(loc):
    print(loc)

def debuggee():
    x = 11
    peforth.ok('11>', loc=dict(locals()))
    x = 22
    peforth.ok('22>', loc=dict(locals()))
    x = 33
    peforth.ok('33>', loc=dict(locals()))

debuggee()    

